Write a function that takes (ra,dec,distance,proper motion ra,proper motion dec,line-of-sight velocity) and their Gaussian uncertainties for a star and computes eccentricity, zmax, rperi, rap and their uncertainties using Monte Carlo sampling with the galpy function galpy.actionAngle.actionAngleStaeckel.EccZmaxRperiRap. Apply to some stars in the Gaia RV sample.

In [26]:
import numpy as np
import galpy.actionAngle as g
import astropy.coordinates as coord
import astropy.units as u

In [27]:
c1 = coord.ICRS(ra=89.014303*u.degree, dec=13.924912*u.degree,
                distance=(37.59*u.mas).to(u.pc, u.parallax()),
                pm_ra_cosdec=372.72*u.mas/u.yr,
                pm_dec=-483.69*u.mas/u.yr,
                radial_velocity=0.37*u.km/u.s)

In [ ]:
def EccZmaxRperiRap_with_uncertainties (ra, dec, distance, pm_ra, pm_dec, vr, ra_u, dec_u, distance_u, pm_ra_u, pm_dec_u, vr_u):
    c1 = coord.ICRS(ra*u.degree, dec*u.degree,
                distance=(37.59*u.mas).to(u.pc, u.parallax()),
                pm_ra_cosdec=372.72*u.mas/u.yr,
                pm_dec=-483.69*u.mas/u.yr,
                radial_velocity=0.37*u.km/u.s)